<a href="https://colab.research.google.com/github/TAKSAKI/japanese_bert/blob/main/%E6%97%A5%E6%9C%AC%E8%AA%9E_BERT_%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語 BERT モデルによる文書分類

日本語 BERT モデルをファインチューニングして文書分類します。

## 準備

必要なパッケージや辞書をインストールします。

In [ ]:
!pip install -q transformers["ja"]
!pip install -q datasets

In [ ]:
import pandas as pd
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, pipeline

## 日本語 BERT

日本語 BERT の事前学習モデルは HuggingFace Transformrs で公開されているものが一番使いやすいです。[フリーで使える日本語の主な大規模言語モデルまとめ](https://zenn.dev/hellorusk/articles/ddee520a5e4318)によくまとまっています。

今回は東北大の BERT-base モデル (v2) を使ってみます。これは日本語 Wikipedia データセットに対して MeCab で UniDic を用いて形態素解析されたもので、さらにサブワードとして WordPiece アルゴリズムを用いてサブワード化が行われたものが公開されています。

- https://huggingface.co/cl-tohoku
- https://github.com/cl-tohoku/bert-japanese

## マスク言語モデルによる推論

BERT はマスク言語モデルによって事前学習されています。そのため、事前学習モデルを使えば、文中の任意のトークンをマスクしても予測を行うことができます。

In [ ]:
model_name = "cl-tohoku/bert-base-japanese-v2"

tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

fitb = pipeline('fill-mask', model=model_name, tokenizer=tokenizer)
fitb("私は[MASK]に合格しました。")

loading file https://huggingface.co/cl-tohoku/bert-base-japanese-v2/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/dc462194780c4638f894e183eec0e214c2891e32a0276d8437e019b6cafd4961.49eeeaed2051802f574bb8508bc9b90ddd068743c7b9c17dd1afcc18c5fc82fd
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-v2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-v2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-v2/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/61a610717b862ec3ecfe65906e562e43eabf578f35c8fe23c7822a66de0c9821.8e4e731c1d54dbc728a7894b3ad143b386987a5f85322fdc7685570a9ec962e3
loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese-v2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6ca2de6dee8b1a5ec94e400e942b44d70a86cd90c

[{'score': 0.21545279026031494,
  'sequence': '私 は オーディション に 合格 し まし た 。',
  'token': 15156,
  'token_str': 'オ ー デ ィ シ ョ ン'},
 {'score': 0.14604008197784424,
  'sequence': '私 は 試験 に 合格 し まし た 。',
  'token': 12076,
  'token_str': '試 験'},
 {'score': 0.13602180778980255,
  'sequence': '私 は これ に 合格 し まし た 。',
  'token': 11190,
  'token_str': 'こ れ'},
 {'score': 0.06066875904798508,
  'sequence': '私 は 見事 に 合格 し まし た 。',
  'token': 20156,
  'token_str': '見 事'},
 {'score': 0.05450180917978287,
  'sequence': '私 は 大学 に 合格 し まし た 。',
  'token': 11188,
  'token_str': '大 学'}]

## 文書分類のためのファインチューニング

2値分類タスクでファインチューニングを行います。

### データセット

In [ ]:
df = pd.read_csv("anotation_kai.csv", usecols= ['sakiyama_label','Article'], nrows=50)
df = df.rename(columns={'sakiyama_label': 'label', 'Article': 'text'})
df['label'] = df['label'].replace({True: 1, False: 0})
df.head()

,text,label
0,[FactCheck] ｢3～5歳児全ての幼児教育､保育を無償化｣は本当か？ー臨時国会所信表...,1
1,[FactCheck] ｢CNNが発砲した男の写真を加工し､白人に見せかけた｣との虚偽情報 ...,0
2,[FactCheck] ｢トランプ大統領の所得税が少なかった理由は多額の寄付｣との根拠不明情...,1
3,[FactCheck] ｢ペンシルベニア州 2万1000人の死者名で郵便投票があったと提訴｣...,1
4,[FactCheck] ｢北海道の中国人技能実習生は1万人以上｣｢占冠村人口の3割以上が中国...,1


In [ ]:
from datasets import Dataset
def encode(examples):
    return tokenizer(examples['text'], padding=True)

dataset = Dataset.from_pandas(df)
dataset = dataset.map(encode)
train_test_data = dataset.train_test_split(test_size=0.8)
dev_data = train_test_data["test"].train_test_split(test_size=0.1)

  0%|          | 0/50 [00:00<?, ?ex/s]

### モデル

In [ ]:
model_name = "cl-tohoku/bert-base-japanese-v2"

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese-v2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6ca2de6dee8b1a5ec94e400e942b44d70a86cd90c211ff7a7123acec2a4b1cdd.6a0ed48d70c8b9e5a276fc6ffdf424ecfb6598f05121c90cf52a781084a8cde4
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32768
}

loading weights file https://huggingface.co/cl-tohoku/bert-base-japanese-v2/res

### ハイパーパラメータ

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


### 評価尺度

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=10)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_data["train"],
    eval_dataset=dev_data["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

RuntimeError: ignored

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset=dev_data["test"])

## ファインチューニングしたモデルで推論

ファインチューニングしたモデルは GPU にあるので、CPU に転送してから推論します。

In [ ]:
classifier = pipeline("sentiment-analysis", model=model.to("cpu"), tokenizer=tokenizer)

In [ ]:
classifier("私の隣室には日本人女性が住んでいますが、彼女も同じ日に発熱して病院に行きました。 彼女と私は一緒に治療を受けているため、お互いに接触している可能性が高いです。")

In [ ]:
for i in range(len(df)):
  print(classifier(df["text"][i]))
  print(df["label"][i], df["text"][i], "\n")